In [1]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *
'''
    ISOT
'''



data = pd.read_csv(r"../data/Des_fake_news/ISOT_PROCESSED.csv")



In [27]:
'''
    Define filtered dataset, classes, and features
'''

'''Drop NA'''
filtered = data.dropna()
classes = filtered["flag"].to_numpy()

#just tweet sentiment classifier
tweet_features = filtered[["text_NN_tweets", "title_NN_tweets", "text_log_tweets", "title_log_tweets", "text_tb_sub_class", "title_tb_sub_class"]].to_numpy()

#just imbd sentiment classifier
imdb_features = filtered[["text_NN_imdb", "title_NN_imdb", "text_log_imdb", "title_log_imdb"]].to_numpy()

# both sentiment classifiers
tweet_and_imdb = filtered[["text_NN_tweets", "title_NN_tweets", "text_log_tweets", "title_log_tweets", "text_tb_sub_class", "title_tb_sub_class",
                           "text_NN_imdb", "title_NN_imdb", "text_log_imdb", "title_log_imdb"]].to_numpy()

# raw polarity and subjectivity scores from Textblob, Vader
raw_features = filtered[["text_tb_pol",	"text_tb_sub",	"title_tb_pol",	"title_tb_sub",	"title_vader_comp",	"title_vader_neg",	
              "title_vader_neu",	"title_vader_pos",	"text_vader_comp",	"text_vader_neg",	"text_vader_neu",	"text_vader_pos"]].to_numpy()

# dataframe to store accuracies for NN and log regression
accuracy_df = pd.DataFrame(columns=["tweet_classifier",
               "imdb_classifier",
               "combined_classifier",
               "raw_sentiments"])


In [28]:
'''
  Logistic Regression outputs
'''


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from openpyxl import load_workbook, Workbook




# average accuracies
logOutput = {}
logMatrices = {"tweet_classifier" : [],
               "imdb_classifier" : [],
               "combined_classifier": [],
               "raw_sentiments" : []}

# sums
tweet_scores = 0
imdb_scores = 0
tweet_imdb_scores = 0
raw_scores = 0

# log regression model, LBFGS with L2 penalty
lbgfs = LogisticRegression(penalty="l2", solver="lbfgs")

# run 100 iterations
for i in range(1):
  print(f"Iteration {i}")
  #tweet_accuracy
  x_train, x_test, y_train, y_test = train_test_split(tweet_features, classes, test_size=0.15, random_state=i)
  lbgfs.fit(x_train, y_train)
  y_pred = lbgfs.predict(x_test)
  tweet_scores += accuracy_score(y_test, y_pred)
  logMatrices["tweet_classifier"].append(confusion_matrix(y_test, y_pred))

  x_train, x_test, y_train, y_test = train_test_split(imdb_features, classes, test_size=0.15, random_state=i)
  lbgfs.fit(x_train, y_train)
  y_pred = lbgfs.predict(x_test)
  imdb_scores += accuracy_score(y_test, y_pred)
  logMatrices["imdb_classifier"].append(confusion_matrix(y_test, y_pred))

  x_train, x_test, y_train, y_test = train_test_split(tweet_and_imdb, classes, test_size=0.15, random_state=i)
  lbgfs.fit(x_train, y_train)
  y_pred = lbgfs.predict(x_test)
  tweet_imdb_scores += accuracy_score(y_test, y_pred)
  logMatrices["combined_classifier"].append(confusion_matrix(y_test, y_pred))

  x_train, x_test, y_train, y_test = train_test_split(raw_features, classes, test_size=0.15, random_state=i)
  lbgfs.fit(x_train, y_train)
  y_pred = lbgfs.predict(x_test)
  raw_scores += accuracy_score(y_test, lbgfs.predict(x_test))
  logMatrices["raw_sentiments"].append(confusion_matrix(y_test, y_pred))

logOutput["tweet_classifier"] = tweet_scores / 1
logOutput["imdb_classifier"] = imdb_scores / 1
logOutput["combined_classifier"] = tweet_imdb_scores / 1
logOutput["raw_sentiments"] = raw_scores / 1

logMatrices["tweet_classifier"] = np.mean(np.array(logMatrices["tweet_classifier"]), axis=0)
logMatrices["imdb_classifier"] = np.mean(np.array(logMatrices["imdb_classifier"]), axis=0)
logMatrices["combined_classifier"] = np.mean(np.array(logMatrices["combined_classifier"]), axis=0)
logMatrices["raw_sentiments"] = np.mean(np.array(logMatrices["raw_sentiments"]), axis=0)

# overwrite book if exists
book = Workbook()
book.save(filename=r"../data/Des_fake_news/ISOT_RESULTS.xlsx")

# write confusion matrices and save
book = load_workbook(r"../data/Des_fake_news/ISOT_RESULTS.xlsx")
writer = pd.ExcelWriter(r"../data/Des_fake_news/ISOT_RESULTS.xlsx", engine="openpyxl")
writer.book = book

for i in logMatrices.keys():
  pd.DataFrame(logMatrices[i]).to_excel(writer, sheet_name=f"matrix_log_{i}")

book.save(filename=r"../data/Des_fake_news/ISOT_RESULTS.xlsx")

accuracy_df.loc["Log Regression"] = logOutput
accuracy_df






Iteration 0


C:\Users\desmo\AppData\Local\Temp\ipykernel_6564\3418070271.py:76: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book


,tweet_classifier,imdb_classifier,combined_classifier,raw_sentiments
Log Regression,0.640172,0.525776,0.640024,0.709404


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split


nnOutputs = {}
nnMatrices = {"tweet_classifier" : [],
               "imdb_classifier" : [],
               "combined_classifier": [],
               "raw_sentiments" : []}


model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(len(tweet_features), 1)),
  tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


x_train, x_test, y_train, y_test = train_test_split(tweet_features, classes, test_size=0.15, random_state=42)
model.fit(x_train, y_train, epochs=10, verbose=2)
nnOutputs["tweet_classifier"] = model.

c:\Users\desmo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1192/1192 - 2s - 2ms/step - accuracy: 0.6475 - loss: 0.6235
Epoch 2/10
1192/1192 - 1s - 1ms/step - accuracy: 0.7199 - loss: 0.5606
Epoch 3/10
1192/1192 - 1s - 1ms/step - accuracy: 0.7340 - loss: 0.5410
Epoch 4/10
1192/1192 - 1s - 1ms/step - accuracy: 0.7437 - loss: 0.5273
Epoch 5/10
1192/1192 - 1s - 1ms/step - accuracy: 0.7500 - loss: 0.5156
Epoch 6/10
1192/1192 - 1s - 962us/step - accuracy: 0.7548 - loss: 0.5064
Epoch 7/10
1192/1192 - 1s - 1ms/step - accuracy: 0.7597 - loss: 0.4993
Epoch 8/10
1192/1192 - 1s - 1ms/step - accuracy: 0.7642 - loss: 0.4935
Epoch 9/10
1192/1192 - 3s - 2ms/step - accuracy: 0.7669 - loss: 0.4897
Epoch 10/10
1192/1192 - 1s - 1ms/step - accuracy: 0.7684 - loss: 0.4859
